Copyright (c) 2019 Computational-Medicine

# Diffusion MRI - Exploration

**Diffusion Imaging In Python**<br>

[DIPY](http://nipy.org/dipy) is a free and open source software project for computational neuroanatomy, focusing mainly on diffusion magnetic resonance imaging (dMRI) analysis. It implements a broad range of algorithms for denoising, registration, reconstruction, tracking, clustering, visualization, and statistical analysis of MRI data.

https://nipy.org/dipy/documentation.html#documentation

### Getting started

Here is a quick snippet showing how to calculate color FA also known as the DEC map. We use a Tensor model to reconstruct the datasets which are saved in a Nifti file along with the b-values and b-vectors which are saved as text files. Finally, we save our result as a Nifti file

In [7]:
import os
import pathlib
import pandas as pd
import numpy as np
home = os.path.expanduser('~')  # To make path to local home directory
# We assume you have made a directory ~/prj/BMED360/results
import nibabel as nib
from dipy.io.image import load_nifti, save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
import dipy

print('nibabel version:', nib.__version__)
print('dipy version:', dipy.__version__)

nibabel version: 2.4.0
dipy version: 0.16.0


In [9]:
BIDSHOME = '../data/mri/mybids'
subj = 'sub-01'

fdwi = '%s/%s/dwi/%s_dwi.nii.gz' % (BIDSHOME, subj, subj )
fbval = '%s/%s/dwi/%s_dwi.bval' % (BIDSHOME, subj, subj )
fbvec = '%s/%s/dwi/%s_dwi.bvec' % (BIDSHOME, subj, subj )

data, affine = load_nifti(fdwi)
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals, bvecs)

tenmodel = TensorModel(gtab)
tenfit = tenmodel.fit(data)

In [11]:
results_folder = '%s/prj/BMED360/results' % (home)

save_nifti('%s/%s_dwi_colorfa.nii.gz' % (results_folder, subj), tenfit.color_fa, affine)